<a href="https://colab.research.google.com/github/KimGriezmann/Bigcontest_2021/blob/master/working_space_JaeHong/pre_function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import tqdm
import os
import matplotlib.pyplot as plt 
import seaborn as sns 

In [2]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
add_hts_2021 = pd.read_csv('/content/drive/MyDrive/bigcontest/add_hts_2021.csv', encoding = 'euc-kr')
test_data = add_hts_2021.head(15)

In [15]:
#누락된 출루율 계산
#타석(PA) 열 추가 / PA = 타수 + 볼넷 + 희비
#1루타(1_hit) 열 추가 / 1_hit = hit - 2_hit - 3_hit - homerun
#장타(XH) 열 추가 / XH = 2_hit + 3_hit + homerun

test_data['avg'] = round(test_data['hit'] / test_data['ab'], 3)
test_data['obp'] = round((test_data['hit'] + test_data['bb'] + test_data['hpp']) / (test_data['ab'] + test_data['bb'] + test_data['hpp'] + test_data['sf']), 3)
test_data['slg'] = round(test_data['tb'] / test_data['ab'], 3)
test_data['ops'] = round(test_data['obp'] + test_data['slg'], 3)

test_data.insert(8, 'pa', test_data['ab'] + test_data['bb'] + test_data['sf'])
test_data.insert(12, '1_hit', test_data['hit'] - test_data['2_hit'] - test_data['3_hit'] - test_data['homerun'])
test_data.insert(16, 'xh', test_data['2_hit'] + test_data['3_hit'] + test_data['homerun'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pan

In [16]:
test_data

,name,Team,DATE,vs,result,bat_order,position,start_member,pa,ab,score,hit,1_hit,2_hit,3_hit,homerun,xh,tb,rbi,sb,cs,bb,hpp,ibb,so,gdp,sh,sf,avg,obp,slg,ops,pitch,avLI,RE24,WPA,GYEAR,PCODE,POSITION,AGE_VA,MONEY,G_ID,power_barrel,contact_barrel,clutch_barrel
0,강백호,kt,20210404,한화,W 3:2,4,1B,1,4,3,0,2,2,0,0,0,0,2,0,2,0,1,0,0,0,0,0,0,0.667,0.75,0.667,1.417,16,1.14,0.89,0.122,2021,68050,내,21,31000만원,0.0,0.0,0.0,0.0
1,강백호,kt,20210406,LG,L 2:3,4,1B,1,4,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0.000,0.00,0.000,0.000,14,2.07,-1.00,-0.183,2021,68050,내,21,31000만원,0.0,0.0,0.0,0.0
2,강백호,kt,20210407,LG,W 7:3,4,1B,1,5,4,0,3,3,0,0,0,0,3,1,0,0,1,0,0,0,0,0,0,0.750,0.80,0.750,1.550,17,1.43,1.75,0.330,2021,68050,내,21,31000만원,0.0,0.0,1.0,0.0
3,강백호,kt,20210408,LG,L 3:7,4,1B,1,4,4,0,2,2,0,0,0,0,2,0,0,0,0,0,0,1,1,0,0,0.500,0.50,0.500,1.000,17,0.89,-0.76,-0.102,2021,68050,내,21,31000만원,0.0,0.0,2.0,0.0
4,강백호,kt,20210409,@삼성,L 5:7,4,1B,1,5,5,0,3,2,0,1,0,1,5,3,0,0,0,0,0,1,1,0,0,0.600,0.60,1.000,1.600,21,0.94,1.88,-0.122,2021,68050,내,21,31000만원,0.0,0.0,0.0,0.0
5,강백호,kt,20210410,@삼성,L 6:7,4,RF,1,4,3,1,1,1,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0.333,0.50,0.333,0.833,22,0.94,1.00,0.103,2021,68050,내,21,31000만원,0.0,0.0,1.0,0.0
6,강백호,kt,20210411,@삼성,L 2:4,4,RF,1,4,4,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0.250,0.25,0.250,0.500,16,1.57,-0.20,-0.024,2021,68050,내,21,31000만원,0.0,0.0,0.0,0.0
7,강백호,kt,20210413,@두산,W 8:7,3,1B,1,5,5,0,1,1,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0.200,0.20,0.200,0.400,18,1.31,-2.23,-0.100,2021,68050,내,21,31000만원,0.0,0.0,1.0,0.0
8,강백호,kt,20210414,@두산,L 1:3,3,1B,1,4,3,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0.000,0.25,0.000,0.250,23,1.19,-0.30,-0.042,2021,68050,내,21,31000만원,0.0,0.0,1.0,0.0
9,강백호,kt,20210415,@두산,W 8:3,3,1B,1,5,2,0,2,1,1,0,0,1,3,0,0,0,3,0,2,0,0,0,0,1.000,1.00,1.500,2.500,18,1.11,1.51,0.134,2021,68050,내,21,31000만원,0.0,1.0,0.0,0.0


In [17]:
def cal_feature(data):
  babip = round((data['hit'] - data['homerun']) / (data['ab'] - data['so'] - data['homerun'] + data['sf']), 3) #babip
  pitch_pa = round(data['pitch'] / data['pa'], 2) #타석당 투구수
  cq = round((0.5 * data['hit'] + 0.3 * data['tb']) / (data['ab'] - data['so']), 3) #컨택 퀄리티
  b_k = round(data['bb'] / data['so'], 3) #볼삼비
  IsoP = data['slg'] - data['avg'] #순수장타율(장타율 - 타율) 
  IsoD = data['obp'] - data['avg'] #순수출루율(출루율 - 타율)
  psn = round((2 * data['homerun'] * data['sb']) / (data['homerun'] + data['sb']), 3) #호타준족수치
  hr_p = round(data['homerun'] / data['pa'], 3) #타석당 홈런비율
  kk_p = round(data['so'] / data['pa'], 3) #타석당 삼진비율
  bb_p = round(data['bb'] / data['pa'], 3) #타석당 볼넷비율
  xh_ab = round(data['xh'] / data['pa'], 3) #타석당 장타비율
  xh_h = round(data['xh'] / data['hit'], 3) #장타비율
  hr_xh = round(data['homerun'] / data['xh'], 3) #장타중 홈런비율
  pa_hr = round(data['pa'] / data['homerun'], 3) #홈런당 타석
  ab_hr = round(data['ab'] / data['homerun'], 3) #홈런당 타수
  ueqr = round((data['hit'] + data['tb'] + 1.5*(data['bb'] + data['hpp'] + data['sb']) + data['sh'] + data['sf'] - data['ibb']/2) / (data['ab'] + data['bb'] + data['hpp'] + data['sh'] + data['sf'] + data['cs'] + data['sb']), 3) #득점창출력
  woba = round((0.72 * data['bb'] + 0.75 * data['hpp'] + 0.9 * data['1_hit'] + 1.24 * data['2_hit'] + 1.56 * data['3_hit'] + 1.95 * data['homerun']) / (data['pa'] - data['ibb']), 3) #조정된 가중출루율

  data['babip'] = babip
  data['pitch/pa'] = pitch_pa
  data['b/k'] = b_k
  data['IsoP'] = IsoP
  data['IsoD'] = IsoD
  data['psn'] = psn
  data['hr%'] = hr_p
  data['kk%'] = kk_p
  data['bb%'] = bb_p
  data['xh/ab'] = xh_ab
  data['xh/h'] = xh_h
  data['hr/xh'] = hr_xh
  data['pa/hr'] = pa_hr
  data['ab/hr'] = ab_hr
  data['ueqr'] = ueqr
  data['wOBA'] = woba

  return data

In [18]:
test = cal_feature(test_data)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

In [19]:
test.iloc[:,7:]

,start_member,pa,ab,score,hit,1_hit,2_hit,3_hit,homerun,xh,tb,rbi,sb,cs,bb,hpp,ibb,so,gdp,sh,sf,avg,obp,slg,ops,pitch,avLI,RE24,WPA,GYEAR,PCODE,POSITION,AGE_VA,MONEY,G_ID,power_barrel,contact_barrel,clutch_barrel,babip,pitch/pa,b/k,IsoP,IsoD,psn,hr%,kk%,bb%,xh/ab,xh/h,hr/xh,pa/hr,ab/hr,ueqr,wOBA
0,1,4,3,0,2,2,0,0,0,0,2,0,2,0,1,0,0,0,0,0,0,0.667,0.75,0.667,1.417,16,1.14,0.89,0.122,2021,68050,내,21,31000만원,0.0,0.0,0.0,0.0,0.667,4.00,inf,0.00,0.083,0.0,0.0,0.00,0.25,0.00,0.000,NaN,inf,inf,1.417,0.630
1,1,4,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0.000,0.00,0.000,0.000,14,2.07,-1.00,-0.183,2021,68050,내,21,31000만원,0.0,0.0,0.0,0.0,0.000,3.50,0.0,0.00,0.000,NaN,0.0,0.50,0.00,0.00,NaN,NaN,inf,inf,0.000,0.000
2,1,5,4,0,3,3,0,0,0,0,3,1,0,0,1,0,0,0,0,0,0,0.750,0.80,0.750,1.550,17,1.43,1.75,0.330,2021,68050,내,21,31000만원,0.0,0.0,1.0,0.0,0.750,3.40,inf,0.00,0.050,NaN,0.0,0.00,0.20,0.00,0.000,NaN,inf,inf,1.500,0.684
3,1,4,4,0,2,2,0,0,0,0,2,0,0,0,0,0,0,1,1,0,0,0.500,0.50,0.500,1.000,17,0.89,-0.76,-0.102,2021,68050,내,21,31000만원,0.0,0.0,2.0,0.0,0.667,4.25,0.0,0.00,0.000,NaN,0.0,0.25,0.00,0.00,0.000,NaN,inf,inf,1.000,0.450
4,1,5,5,0,3,2,0,1,0,1,5,3,0,0,0,0,0,1,1,0,0,0.600,0.60,1.000,1.600,21,0.94,1.88,-0.122,2021,68050,내,21,31000만원,0.0,0.0,0.0,0.0,0.750,4.20,0.0,0.40,0.000,NaN,0.0,0.20,0.00,0.20,0.333,0.0,inf,inf,1.600,0.672
5,1,4,3,1,1,1,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0.333,0.50,0.333,0.833,22,0.94,1.00,0.103,2021,68050,내,21,31000만원,0.0,0.0,1.0,0.0,0.333,5.50,inf,0.00,0.167,NaN,0.0,0.00,0.25,0.00,0.000,NaN,inf,inf,0.875,0.405
6,1,4,4,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0.250,0.25,0.250,0.500,16,1.57,-0.20,-0.024,2021,68050,내,21,31000만원,0.0,0.0,0.0,0.0,0.250,4.00,NaN,0.00,0.000,NaN,0.0,0.00,0.00,0.00,0.000,NaN,inf,inf,0.500,0.225
7,1,5,5,0,1,1,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0.200,0.20,0.200,0.400,18,1.31,-2.23,-0.100,2021,68050,내,21,31000만원,0.0,0.0,1.0,0.0,0.250,3.60,0.0,0.00,0.000,NaN,0.0,0.20,0.00,0.00,0.000,NaN,inf,inf,0.400,0.180
8,1,4,3,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0.000,0.25,0.000,0.250,23,1.19,-0.30,-0.042,2021,68050,내,21,31000만원,0.0,0.0,1.0,0.0,0.000,5.75,inf,0.00,0.250,NaN,0.0,0.00,0.25,0.00,NaN,NaN,inf,inf,0.375,0.180
9,1,5,2,0,2,1,1,0,0,1,3,0,0,0,3,0,2,0,0,0,0,1.000,1.00,1.500,2.500,18,1.11,1.51,0.134,2021,68050,내,21,31000만원,0.0,1.0,0.0,0.0,1.000,3.60,inf,0.50,0.000,NaN,0.0,0.00,0.60,0.20,0.500,0.0,inf,inf,1.700,1.433


In [20]:
def make_percentage(data):
  data['hit'] = round(data['hit'] / data['pa'], 3)
  data['1_hit'] = round(data['1_hit'] / data['pa'], 3)
  data['xh'] = round(data['xh'] / data['pa'], 3)
  data['2_hit'] = round(data['2_hit'] / data['pa'], 3)
  data['3_hit'] = round(data['3_hit'] / data['pa'], 3)
  data['homerun'] = round(data['homerun'] / data['pa'], 3)
  data['tb'] = round(data['tb'] / data['pa'], 3)
  data['rbi'] = round(data['rbi'] / data['pa'], 3)
  data['sb'] = round(data['sb'] / data['pa'], 3)
  data['cs'] = round(data['cs'] / data['pa'], 3)
  data['bb'] = round(data['bb'] / data['pa'], 3)
  data['hpp'] = round(data['hpp'] / data['pa'], 3)
  data['ibb'] = round(data['ibb'] / data['pa'], 3)
  data['so'] = round(data['so'] / data['pa'], 3)
  data['gdp'] = round(data['gdp'] / data['pa'], 3)
  data['sh'] = round(data['sh'] / data['pa'], 3)
  data['sf'] = round(data['sf'] / data['pa'], 3)
  data['avLI'] = round(data['avLI'] / data['pa'], 3)
  data['RE24'] = round(data['RE24'] / data['pa'], 3)
  data['WPA'] = round(data['WPA'] / data['pa'], 3)
  data['power_barrel'] = round(data['power_barrel'] / data['pa'], 3)
  data['contact_barrel'] = round(data['contact_barrel'] / data['pa'], 3)
  data['clutch_barrel'] = round(data['clutch_barrel'] / data['pa'], 3)

  return data


In [21]:
test2 = make_percentage(test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [22]:
test2.iloc[:,7:]

,start_member,pa,ab,score,hit,1_hit,2_hit,3_hit,homerun,xh,tb,rbi,sb,cs,bb,hpp,ibb,so,gdp,sh,sf,avg,obp,slg,ops,pitch,avLI,RE24,WPA,GYEAR,PCODE,POSITION,AGE_VA,MONEY,G_ID,power_barrel,contact_barrel,clutch_barrel,babip,pitch/pa,b/k,IsoP,IsoD,psn,hr%,kk%,bb%,xh/ab,xh/h,hr/xh,pa/hr,ab/hr,ueqr,wOBA
0,1,4,3,0,0.50,0.50,0.00,0.0,0.0,0.00,0.50,0.00,0.5,0.0,0.25,0.0,0.0,0.00,0.00,0.0,0.0,0.667,0.75,0.667,1.417,16,0.285,0.222,0.030,2021,68050,내,21,31000만원,0.0,0.00,0.00,0.00,0.667,4.00,inf,0.00,0.083,0.0,0.0,0.00,0.25,0.00,0.000,NaN,inf,inf,1.417,0.630
1,1,4,4,0,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.50,0.00,0.0,0.0,0.000,0.00,0.000,0.000,14,0.518,-0.250,-0.046,2021,68050,내,21,31000만원,0.0,0.00,0.00,0.00,0.000,3.50,0.0,0.00,0.000,NaN,0.0,0.50,0.00,0.00,NaN,NaN,inf,inf,0.000,0.000
2,1,5,4,0,0.60,0.60,0.00,0.0,0.0,0.00,0.60,0.20,0.0,0.0,0.20,0.0,0.0,0.00,0.00,0.0,0.0,0.750,0.80,0.750,1.550,17,0.286,0.350,0.066,2021,68050,내,21,31000만원,0.0,0.00,0.20,0.00,0.750,3.40,inf,0.00,0.050,NaN,0.0,0.00,0.20,0.00,0.000,NaN,inf,inf,1.500,0.684
3,1,4,4,0,0.50,0.50,0.00,0.0,0.0,0.00,0.50,0.00,0.0,0.0,0.00,0.0,0.0,0.25,0.25,0.0,0.0,0.500,0.50,0.500,1.000,17,0.222,-0.190,-0.026,2021,68050,내,21,31000만원,0.0,0.00,0.50,0.00,0.667,4.25,0.0,0.00,0.000,NaN,0.0,0.25,0.00,0.00,0.000,NaN,inf,inf,1.000,0.450
4,1,5,5,0,0.60,0.40,0.00,0.2,0.0,0.20,1.00,0.60,0.0,0.0,0.00,0.0,0.0,0.20,0.20,0.0,0.0,0.600,0.60,1.000,1.600,21,0.188,0.376,-0.024,2021,68050,내,21,31000만원,0.0,0.00,0.00,0.00,0.750,4.20,0.0,0.40,0.000,NaN,0.0,0.20,0.00,0.20,0.333,0.0,inf,inf,1.600,0.672
5,1,4,3,1,0.25,0.25,0.00,0.0,0.0,0.00,0.25,0.25,0.0,0.0,0.25,0.0,0.0,0.00,0.00,0.0,0.0,0.333,0.50,0.333,0.833,22,0.235,0.250,0.026,2021,68050,내,21,31000만원,0.0,0.00,0.25,0.00,0.333,5.50,inf,0.00,0.167,NaN,0.0,0.00,0.25,0.00,0.000,NaN,inf,inf,0.875,0.405
6,1,4,4,0,0.25,0.25,0.00,0.0,0.0,0.00,0.25,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.250,0.25,0.250,0.500,16,0.392,-0.050,-0.006,2021,68050,내,21,31000만원,0.0,0.00,0.00,0.00,0.250,4.00,NaN,0.00,0.000,NaN,0.0,0.00,0.00,0.00,0.000,NaN,inf,inf,0.500,0.225
7,1,5,5,0,0.20,0.20,0.00,0.0,0.0,0.00,0.20,0.00,0.0,0.0,0.00,0.0,0.0,0.20,0.20,0.0,0.0,0.200,0.20,0.200,0.400,18,0.262,-0.446,-0.020,2021,68050,내,21,31000만원,0.0,0.00,0.20,0.00,0.250,3.60,0.0,0.00,0.000,NaN,0.0,0.20,0.00,0.00,0.000,NaN,inf,inf,0.400,0.180
8,1,4,3,0,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.25,0.0,0.0,0.00,0.00,0.0,0.0,0.000,0.25,0.000,0.250,23,0.298,-0.075,-0.010,2021,68050,내,21,31000만원,0.0,0.00,0.25,0.00,0.000,5.75,inf,0.00,0.250,NaN,0.0,0.00,0.25,0.00,NaN,NaN,inf,inf,0.375,0.180
9,1,5,2,0,0.40,0.20,0.20,0.0,0.0,0.20,0.60,0.00,0.0,0.0,0.60,0.0,0.4,0.00,0.00,0.0,0.0,1.000,1.00,1.500,2.500,18,0.222,0.302,0.027,2021,68050,내,21,31000만원,0.0,0.20,0.00,0.00,1.000,3.60,inf,0.50,0.000,NaN,0.0,0.00,0.60,0.20,0.500,0.0,inf,inf,1.700,1.433
